<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [126]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [127]:
DATA_PATH = '/workspace/data/criteo/dac'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

PREPROCESSING_TRANSFORMER_PATH = os.path.join(DATA_PATH, 'preprocessing_transformer')

PREDICTIONS_PATH = os.path.join(DATA_PATH, 'predictions.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [128]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [129]:
df_test = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

In [130]:
# df = df.sample(False, 0.5)

In [131]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [132]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [133]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [134]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [135]:
index_columns = [column_name + "_index" for column_name in cat_columns]
vector_columns = [column_name + "_vector" for column_name in cat_columns]

In [136]:
string_indexers = [StringIndexer(inputCol=column_name, outputCol=output_column, handleInvalid='keep') 
                   for column_name, output_column in zip(cat_columns, index_columns)]
one_hot_encoder = OneHotEncoderEstimator(inputCols=index_columns, outputCols=vector_columns)
vector_assembler = VectorAssembler(inputCols=num_columns + vector_columns, outputCol='features')
pipeline_stages = string_indexers + [one_hot_encoder, vector_assembler]
preprocessing_pipeline = Pipeline(stages=pipeline_stages)

### Fit and save pipeline

In [137]:
preprocessing_pipeline_transformer = preprocessing_pipeline.fit(df)

In [138]:
preprocessing_pipeline_transformer.write().overwrite().save(PREPROCESSING_TRANSFORMER_PATH)

### Load fitted pipeline

In [139]:
from pyspark.ml import PipelineModel

preprocessing_pipeline_transformer = PipelineModel.load(PREPROCESSING_TRANSFORMER_PATH)

### Transform dataset using pipeline

In [140]:
df = preprocessing_pipeline_transformer.transform(df)
df = df.select('id', 'features', F.col('_c0').alias('label'))

In [141]:
df_test = df_test.fillna(0, subset=num_columns)
df_test = preprocessing_pipeline_transformer.transform(df_test)
df_test = df_test.select('id', 'features')

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [142]:
from pyspark.sql import Window

In [143]:
def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    parts = []
    
    
    tmp = df\
        .withColumn('rank', F.percent_rank().over(
            Window().orderBy(F.col(split_col))))
    
    cumulative_sums = [sum(parts_fractions[0:i]) for i in range(len(parts_fractions) + 1)]
    for i in range(len(parts_fractions)):
        samples = tmp\
            .filter((cumulative_sums[i] <= F.col('rank')) & (F.col('rank') < cumulative_sums[i + 1]))\
            .drop('rank')
        parts.append(samples)
    return parts

In [144]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [145]:
N = df.count()

In [146]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999997817148535, 0.09999997271435669, 0.09999997271435669)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [147]:
from pyspark.ml.classification import LogisticRegression

In [159]:
# lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr = LogisticRegression(maxIter=10, regParam=0)
lr_model = lr.fit(train_df)

In [160]:
lr_model.transform(val_df)\
    .select("label", "probability")\
    .take(10)

[Row(label=0, probability=DenseVector([0.7234, 0.2766])),
 Row(label=1, probability=DenseVector([0.8302, 0.1698])),
 Row(label=1, probability=DenseVector([0.26, 0.74])),
 Row(label=1, probability=DenseVector([0.7691, 0.2309])),
 Row(label=1, probability=DenseVector([0.5927, 0.4073])),
 Row(label=1, probability=DenseVector([0.7023, 0.2977])),
 Row(label=1, probability=DenseVector([0.648, 0.352])),
 Row(label=0, probability=DenseVector([0.8063, 0.1937])),
 Row(label=1, probability=DenseVector([0.7306, 0.2694])),
 Row(label=1, probability=DenseVector([0.6933, 0.3067]))]

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [161]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def get_probability(model, df):
    return model.transform(df)\
        .select("probability", "label")\
        .rdd\
        .map(lambda row: (float(row['probability'][1]), float(row['label'])))

def cross_entropy(probability, label):
    return -label * np.log(probability) - (1 - label) * np.log(1 - probability)
    
    
def rocauc(model, df):
    probabilities = get_probability(model, df)
    return BinaryClassificationMetrics(probabilities).areaUnderROC

def logloss(model, df):        
    probabilities = get_probability(model, df)
    return predictions\
        .map(lambda data:  cross_entropy(data[0], data[1]))\
        .mean()

def ne(model, df):
    probabilities = get_probability(model, df)
    label_mean = probabilities\
        .map(lambda data: data[1])\
        .mean()
    best_loss = cross_entropy(label_mean, label_mean)
    real_loss = probabilities\
        .map(lambda data:  cross_entropy(data[0], data[1]))\
        .mean()
    return real_loss / best_loss

In [162]:
rocauc(lr_model, val_df)

0.7033946931669183

In [163]:
ne(lr_model, val_df)

0.9180212864261902

In [164]:
rocauc(lr_model, test_df)

0.7021411416137866

In [165]:
ne(lr_model, test_df)

0.9190380466919819

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [166]:
import pandas

In [167]:
df_predictions = lr_model.transform(df_test).select('id', 'probability').toPandas()
df_predictions['proba'] = df_predictions['probability'].map(lambda p: p[1] if not pandas.isna(p[1]) else 0)

In [168]:
df_predictions[['id', 'proba']].to_csv(PREDICTIONS_PATH, index=False)

In [169]:
len(df_predictions)

917961